In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:

import tensorflow.keras.applications
for model in dir(tensorflow.keras.applications):
  print(model)

ConvNeXtBase
ConvNeXtLarge
ConvNeXtSmall
ConvNeXtTiny
ConvNeXtXLarge
DenseNet121
DenseNet169
DenseNet201
EfficientNetB0
EfficientNetB1
EfficientNetB2
EfficientNetB3
EfficientNetB4
EfficientNetB5
EfficientNetB6
EfficientNetB7
EfficientNetV2B0
EfficientNetV2B1
EfficientNetV2B2
EfficientNetV2B3
EfficientNetV2L
EfficientNetV2M
EfficientNetV2S
InceptionResNetV2
InceptionV3
MobileNet
MobileNetV2
MobileNetV3Large
MobileNetV3Small
NASNetLarge
NASNetMobile
RegNetX002
RegNetX004
RegNetX006
RegNetX008
RegNetX016
RegNetX032
RegNetX040
RegNetX064
RegNetX080
RegNetX120
RegNetX160
RegNetX320
RegNetY002
RegNetY004
RegNetY006
RegNetY008
RegNetY016
RegNetY032
RegNetY040
RegNetY064
RegNetY080
RegNetY120
RegNetY160
RegNetY320
ResNet101
ResNet101V2
ResNet152
ResNet152V2
ResNet50
ResNet50V2
ResNetRS101
ResNetRS152
ResNetRS200
ResNetRS270
ResNetRS350
ResNetRS420
ResNetRS50
VGG16
VGG19
Xception
__builtins__
__cached__
__doc__
__file__
__loader__
__name__
__package__
__path__
__spec__
convnext
densenet
efficie

In [3]:
help(tensorflow.keras.applications)

Help on package tensorflow.keras.applications in tensorflow.keras:

NAME
    tensorflow.keras.applications - AUTOGENERATED. DO NOT EDIT.

PACKAGE CONTENTS
    convnext (package)
    densenet (package)
    efficientnet (package)
    efficientnet_v2 (package)
    imagenet_utils (package)
    inception_resnet_v2 (package)
    inception_v3 (package)
    mobilenet (package)
    mobilenet_v2 (package)
    mobilenet_v3 (package)
    nasnet (package)
    regnet (package)
    resnet (package)
    resnet50 (package)
    resnet_rs (package)
    resnet_v2 (package)
    vgg16 (package)
    vgg19 (package)
    xception (package)

FILE
    /usr/local/lib/python3.10/dist-packages/keras/api/_v2/keras/applications/__init__.py




In [4]:
#include_top helps decide whether to include fully connected layer of the model, we do not want to include it so defined as False
base_model = MobileNetV2(weights='imagenet', include_top =False)
base_model.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, None, None, 32)       864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, None, None, 32)       128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReL

In [5]:
input_shape = (299, 299, 3)

input_tensor = tf.keras.Input(shape=input_shape)
rotate = tf.keras.layers.RandomRotation(0.4)(input_tensor)
width = tf.keras.layers.RandomWidth(0.2)(rotate)
height = tf.keras.layers.RandomHeight(0.2)(width)
zoom = tf.keras.layers.RandomZoom(0.2)(height)
flip = tf.keras.layers.RandomFlip('horizontal_and_vertical')(zoom)
baseoutput = base_model(flip)
globavg = GlobalAveragePooling2D()(baseoutput)
dropout1 = Dropout(0.4)(globavg)
bn1 = BatchNormalization()(dropout1)
dense1 = Dense(256, activation='relu')(bn1)
dropout2 = Dropout(0.4)(dense1)
predictions = Dense(1, activation='sigmoid')(dropout2)

model = Model(inputs=input_tensor, outputs=predictions)

for layer in base_model.layers:
  layer.trainable=False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 random_rotation (RandomRot  (None, 299, 299, 3)       0         
 ation)                                                          
                                                                 
 random_width (RandomWidth)  (None, 299, None, 3)      0         
                                                                 
 random_height (RandomHeigh  (None, None, None, 3)     0         
 t)                                                              
                                                                 
 random_zoom (RandomZoom)    (None, None, None, 3)     0         
                                                                 
 random_flip (RandomFlip)    (None, None, None, 3)     0     

In [7]:
split = ['train[:80%]', 'train[20%:]']
(train_dataset, val_dataset), info = tfds.load('cats_vs_dogs', split=split, with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.1.incompleteQG49II/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [2]:
help(tfds)

Help on package tensorflow_datasets:

NAME
    tensorflow_datasets - `tensorflow_datasets` (`tfds`) defines a collection of datasets ready-to-use with TensorFlow.

DESCRIPTION
    Each dataset is defined as a `tfds.core.DatasetBuilder`, which encapsulates
    the logic to download the dataset and construct an input pipeline, as well as
    contains the dataset documentation (version, splits, number of examples, etc.).
    
    The main library entrypoints are:
    
    * `tfds.builder`: fetch a `tfds.core.DatasetBuilder` by name
    * `tfds.load`: convenience method to construct a builder, download the data, and
      create an input pipeline, returning a `tf.data.Dataset`.
    
    Documentation:
    
    * These API docs
    * [Available datasets](https://www.tensorflow.org/datasets/catalog/overview)
    * [Colab tutorial](https://colab.research.google.com/github/tensorflow/datasets/blob/master/docs/overview.ipynb)
    * [Add a dataset](https://www.tensorflow.org/datasets/add_dataset

In [4]:
help(tfds.load)

Help on function load in module tensorflow_datasets.core.load:

load(name: 'str', *, split: 'Optional[Tree[splits_lib.SplitArg]]' = None, data_dir: 'Union[None, str, os.PathLike]' = None, batch_size: 'Optional[int]' = None, shuffle_files: 'bool' = False, download: 'bool' = True, as_supervised: 'bool' = False, decoders: 'Optional[TreeDict[decode.partial_decode.DecoderArg]]' = None, read_config: 'Optional[read_config_lib.ReadConfig]' = None, with_info: 'bool' = False, builder_kwargs: 'Optional[Dict[str, Any]]' = None, download_and_prepare_kwargs: 'Optional[Dict[str, Any]]' = None, as_dataset_kwargs: 'Optional[Dict[str, Any]]' = None, try_gcs: 'bool' = False)
    Loads the named dataset into a `tf.data.Dataset`.
    
    `tfds.load` is a convenience method that:
    
    1. Fetch the `tfds.core.DatasetBuilder` by name:
    
       ```python
       builder = tfds.builder(name, data_dir=data_dir, **builder_kwargs)
       ```
    
    2. Generate the data (when `download=True`):
    
       

In [8]:
def preprocess_norm(image, label):
  image = tf.image.resize(image, [299, 299])
  image = image / 127.5 - 1
  return image, label

norm_train = train_dataset.map(preprocess_norm).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
norm_val = val_dataset.map(preprocess_norm).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
preprocess_input

<function keras.src.applications.mobilenet.preprocess_input(x, data_format=None)>

In [10]:
norm_train

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [11]:
#when calling a function and inputting arguments CTRL + SPACE will display which arguments are available

In [12]:
for layer in dir(tf.keras.layers):
  print(layer)

AbstractRNNCell
Activation
ActivityRegularization
Add
AdditiveAttention
AlphaDropout
Attention
Average
AveragePooling1D
AveragePooling2D
AveragePooling3D
AvgPool1D
AvgPool2D
AvgPool3D
BatchNormalization
Bidirectional
CategoryEncoding
CenterCrop
Concatenate
Conv1D
Conv1DTranspose
Conv2D
Conv2DTranspose
Conv3D
Conv3DTranspose
ConvLSTM1D
ConvLSTM2D
ConvLSTM3D
Convolution1D
Convolution1DTranspose
Convolution2D
Convolution2DTranspose
Convolution3D
Convolution3DTranspose
Cropping1D
Cropping2D
Cropping3D
Dense
DenseFeatures
DepthwiseConv1D
DepthwiseConv2D
Discretization
Dot
Dropout
ELU
EinsumDense
Embedding
Flatten
GRU
GRUCell
GaussianDropout
GaussianNoise
GlobalAveragePooling1D
GlobalAveragePooling2D
GlobalAveragePooling3D
GlobalAvgPool1D
GlobalAvgPool2D
GlobalAvgPool3D
GlobalMaxPool1D
GlobalMaxPool2D
GlobalMaxPool3D
GlobalMaxPooling1D
GlobalMaxPooling2D
GlobalMaxPooling3D
GroupNormalization
HashedCrossing
Hashing
Identity
Input
InputLayer
InputSpec
IntegerLookup
LSTM
LSTMCell
Lambda
Layer
L

In [13]:
EPOCHS=1

model.fit(norm_train, epochs=EPOCHS, verbose=1)

582/582 [==============================] - 498s 838ms/step - loss: 0.2349 - acc: 0.9074


In [14]:
for data in norm_val.take(1):
  images, labels = data[0], data[1]
  x = np.expand_dims(images[0].numpy(), axis=0)
  pred = model.predict(x)
  print(pred[0], labels[0])
  x = np.expand_dims(images[1].numpy(), axis=0)
  pred = model.predict(x)
  print(pred[0], labels[1])

1/1 [==============================] - 2s 2s/step
[0.9999937] tf.Tensor(1, shape=(), dtype=int64)
1/1 [==============================] - 0s 22ms/step
[0.00028549] tf.Tensor(0, shape=(), dtype=int64)


In [15]:
# attempted operation maxes out system RAM
#norm_train_list = list(norm_train.as_numpy_iterator())
#norm_train_list[0]